In [ ]:
"""
    DistilBERT training via knowledge distillation from BERT using PyTorch and Hugging Face Transformers.
"""
# !pip install torch transformers datasets

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AdamW 
from tqdm import tqdm

In [ ]:
"""
    AdamW is a popular optimization algorithm in deep learning, especially well-suited for training models like Transformers (e.g., BERT, GPT, etc.). 
    It is a variant of the Adam optimizer that introduces a correct way to apply weight decay (L2 regularization).
    AdamW helps prevent overfitting while maintaining the benefits of Adam (adaptive learning rates, momentum).
    It is the default optimizer in Hugging Face Transformers and many other frameworks for fine-tuning pre-trained language models.
"""

In [ ]:
# Load and Preprocess the Dataset
dataset = load_dataset("imdb", split='train[:1%]').train_test_split(test_size=0.2) # Loads 1% of the IMDb dataset's training set for a quick demo or test
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Loads the tokenizer that corresponds to bert-base-uncased
"""
    The IMDB dataset is a popular dataset used for binary sentiment classification—determining whether a movie review is positive or negative.
"""

In [ ]:
# Tokenizing the Text
